In [83]:
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime
import yfinance as yf
import pandas as pd
import numpy as np
import html5lib
import os
import glob

In [ ]:
# Lecture des fichiers
df_crypto_infos = pd.read_csv("csv/csv_bdd/crypto_infos.csv")

print("\nInfos cryptos :")
display(df_crypto_infos.head())

# Sélection des 2 premières pour test
cryptos_tickers = df_crypto_infos["Ticker_cryptos_Yf"].tolist()
cryptos_nom = df_crypto_infos["Short_Name_Cryptos"].tolist()

historique = {} # dictionnaire pour stocker les futurs df de chaque crypto. clé = ticker, valeur = liste de df (ici une seule valeur par clé)

for ticker, nom in zip(cryptos_tickers, cryptos_nom):
    try:
        crypto = yf.Ticker(ticker)
        df = crypto.history(period="max", interval="1mo")

        if not df.empty:
            df = df.reset_index()  # pour avoir la colonne Date
            df["Ticker_cryptos_Yf"] = ticker
            df["Short_Name_Cryptos"] = nom
            historique[ticker] = df # historique[ticker] représente la clé pour la valeur de chaque df.

            print(f"✅ {nom} : {len(df)} points récupérés (du {df['Date'].min().date()} au {df['Date'].max().date()})")
        else:
            print(f"⚠️ {nom} : aucune donnée disponible")

    except Exception as e:
        print(f"❌ Erreur pour {nom} ({ticker}): {e}")

print(f"\n📊 Total cryptos récupérées : {len(historique)}/{len(cryptos_tickers)}")

# Concaténer tous les historiques en un seul DataFrame

# Concaténation de tous les historiques
df_hist = pd.concat(historique.values(), ignore_index=True)
# Convertir la colonne "Date" en format datetime et reformater en "JJ-MM-AAAA"
df_hist["Date"] = df_hist["Date"].dt.strftime("%Y-%m-%d")
 # Arrondir la colonne "Close"
df_hist["Close"] = df_hist["Close"].round(3)

# Supprimer colonnes inutiles
df_hist = df_hist.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], errors="ignore")

display(df_hist)

# Sauvegarde
df_hist.to_csv("csv/csv_bdd/historique_cryptos.csv", index=False, encoding="utf-8")
print("\n💾 Fichier sauvegardé : csv/csv_bdd/historique_cryptos.csv")




Infos cryptos :


,Unnamed: 0,Short_Name_Cryptos,Ticker_cryptos_Yf,Ticker_Cryptos,Prix_actuel,Capitalisation_Boursiere,Offre_En_Circulation,ATH,MAJ
0,0,Bitcoin,BTC-USD,btc,107484.00,2146070426142,1.994381e+07,126080.00,2025-11-03
1,1,Ethereum,ETH-USD,eth,3698.04,447070715379,1.206979e+08,4946.05,2025-11-03
2,2,Tether,USDT-USD,usdt,1.00,183460057302,1.834536e+11,1.32,2025-11-03
3,3,XRP,XRP-USD,xrp,2.40,144458986627,6.010720e+10,3.65,2025-11-03
4,4,BNB,BNB-USD,bnb,1009.57,139176167606,1.377378e+08,1369.99,2025-11-03


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker_cryptos_Yf,Short_Name_Cryptos
0,2014-09-01 00:00:00+00:00,465.864014,468.174011,372.239990,386.944000,410881000,0.0,0.0,BTC-USD,Bitcoin
1,2014-10-01 00:00:00+00:00,387.427002,411.697998,289.295990,338.321014,902994450,0.0,0.0,BTC-USD,Bitcoin
2,2014-11-01 00:00:00+00:00,338.649994,457.092987,320.626007,378.046997,659733360,0.0,0.0,BTC-USD,Bitcoin
3,2014-12-01 00:00:00+00:00,378.248993,384.037994,304.231995,320.192993,553102310,0.0,0.0,BTC-USD,Bitcoin
4,2015-01-01 00:00:00+00:00,320.434998,320.434998,171.509995,217.464005,1098811912,0.0,0.0,BTC-USD,Bitcoin


✅ Bitcoin : 135 points récupérés (du 2014-09-01 au 2025-11-01)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker_cryptos_Yf,Short_Name_Cryptos
0,2017-11-01 00:00:00+00:00,308.644989,522.307007,294.541992,447.114014,26957496832,0.0,0.0,ETH-USD,Ethereum
1,2017-12-01 00:00:00+00:00,445.209015,881.943970,414.411011,756.732971,79862276544,0.0,0.0,ETH-USD,Ethereum
2,2018-01-01 00:00:00+00:00,755.757019,1432.880005,742.004028,1118.310059,163610220288,0.0,0.0,ETH-USD,Ethereum
3,2018-02-01 00:00:00+00:00,1119.369995,1161.349976,574.419006,855.198975,83393441152,0.0,0.0,ETH-USD,Ethereum
4,2018-03-01 00:00:00+00:00,856.012024,880.302002,368.632996,396.457001,53716179584,0.0,0.0,ETH-USD,Ethereum


...

✅ Binance-Peg WETH : 95 points récupérés (du 2018-01-01 au 2025-11-01)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker_cryptos_Yf,Short_Name_Cryptos
0,2021-11-01 00:00:00+00:00,0.295284,0.333284,0.179293,0.179393,5712576,0.0,0.0,APT-USD,Aptos
1,2021-12-01 00:00:00+00:00,0.179387,0.179968,0.054257,0.080676,1451097,0.0,0.0,APT-USD,Aptos
2,2022-01-01 00:00:00+00:00,0.080680,0.120717,0.047702,0.047802,7262162,0.0,0.0,APT-USD,Aptos
3,2022-02-01 00:00:00+00:00,0.047803,0.051203,0.023607,0.030044,12020899,0.0,0.0,APT-USD,Aptos
4,2022-03-01 00:00:00+00:00,0.030039,0.030189,0.020491,0.021696,12080908,0.0,0.0,APT-USD,Aptos


✅ Aptos : 44 points récupérés (du 2021-11-01 au 2025-06-01)

📊 Total cryptos récupérées : 54/60


,Date,Close,Ticker_cryptos_Yf,Short_Name_Cryptos
0,2014-09-01,386.944,BTC-USD,Bitcoin
1,2014-10-01,338.321,BTC-USD,Bitcoin
2,2014-11-01,378.047,BTC-USD,Bitcoin
3,2014-12-01,320.193,BTC-USD,Bitcoin
4,2015-01-01,217.464,BTC-USD,Bitcoin
...,...,...,...,...
3366,2025-02-01,0.000,APT-USD,Aptos
3367,2025-03-01,0.000,APT-USD,Aptos
3368,2025-04-01,0.000,APT-USD,Aptos
3369,2025-05-01,0.000,APT-USD,Aptos



💾 Fichier sauvegardé : csv/csv_bdd/historique_cryptos.csv
